### <font color="darkblue"><i>A predictive model which tries to predict whether customers will accept a loan offer or not. The model uses only features that will be known at the time of sending an offer. Once the performance metrics of the model have been assessed and the performance (e.g. accuracy) is evaluated to be adequate enough, the model can be used by employees before sending the actual offers. If the model predicts that the offer will not be accepted (i.e. will be refused and/or cancelled) with a high probability, the employee can chose to change the offer and/or not send an offer / send multiple offers. This can reduce the company's overhead/resources in the offer sending process.</i></font>

### <font color="green">imports, preparation and configuration</font>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from IPython.display import HTML
HTML('''<script>
code_show_err=false; 
function code_toggle_err() {
 if (code_show_err){
 $('div.output_stderr').hide();
 } else {
 $('div.output_stderr').show();
 }
 code_show_err = !code_show_err
} 
$( document ).ready(code_toggle_err);
</script>
To toggle on/off output_stderr, click <a href="javascript:code_toggle_err()">here</a>.''')

In [3]:
offers = pd.read_table("../data/laraData/offer_events.csv", sep=";")
applications = pd.read_table("../data/laraData/application_events.csv", sep=";")
all_events = pd.read_table("../data/laraData/events.csv", sep=";")
applications.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (1,3,5,7,10,11,12,29,38,39,41,43,45,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,A_ID,A_DT_CREATED,A_INIT_REQ_AMT,A_USER_CREATED,A_APPLICATIONTYPE,A_APPLICATIONTYPE_DESC,A_LOANGOALTYPE,A_LOANGOALTYPE_DESC,AS_ID,AS_APPLICATIONSTATE,...,NUMBER_OF_CHILDREN,MARITAL_STATUS,SEX,HOUSING_TYPE,SCE_CREDITSCORE,SCE_ACCEPTED,SCE_ACCEPTED_OVERRIDE,SCE_MAX_AMOUNT,SCE_MAX_AMOUNT_OVERRIDE,INCOMEAMOUNT_YEAR
0,719696,"1/1/2017 2:42:29,064000",29000.0,USER_1,520,Nieuw krediet,866.0,Auto,1453642,511,...,0.0,Samenwonend,Vrouw,Koophuis,784.0,1.0,1.0,49340.0,49340.0,41088.0
1,719696,"1/1/2017 2:42:29,064000",29000.0,USER_1,520,Nieuw krediet,866.0,Auto,1454067,513,...,0.0,Samenwonend,Vrouw,Koophuis,784.0,1.0,1.0,49340.0,49340.0,41088.0
2,719696,"1/1/2017 2:42:29,064000",29000.0,USER_1,520,Nieuw krediet,866.0,Auto,1458788,517,...,0.0,Samenwonend,Vrouw,Koophuis,784.0,1.0,1.0,49340.0,49340.0,41088.0
3,719696,"1/1/2017 2:42:29,064000",29000.0,USER_1,520,Nieuw krediet,866.0,Auto,1456650,516,...,0.0,Samenwonend,Vrouw,Koophuis,784.0,1.0,1.0,49340.0,49340.0,41088.0
4,719696,"1/1/2017 2:42:29,064000",29000.0,USER_1,520,Nieuw krediet,866.0,Auto,1453643,512,...,0.0,Samenwonend,Vrouw,Koophuis,784.0,1.0,1.0,49340.0,49340.0,41088.0


In [4]:
offers.head()

,A_ID,O_ID,O_SCENARIO,O_DT_CREATED,O_AMOUNT,O_CREDITSCORE,O_FIRST_WITHDRAWL_AMOUNT,NUMBER_OF_TERMS,O_MONTHLY_COST,O_ACCEPTED,...,NUMBER_OF_CHILDREN,MARITAL_STATUS,SEX,HOUSING_TYPE,INCOMEAMOUNT_YEAR,SCE_CREDITSCORE,SCE_ACCEPTED,SCE_ACCEPTED_OVERRIDE,SCE_MAX_AMOUNT,SCE_MAX_AMOUNT_OVERRIDE
0,719696,210708,1367858,"2/1/2017 10:38:26,536000",29000.0,784.0,26604.0,120,29529,1,...,0,Samenwonend,Vrouw,Koophuis,41088.0,784.0,1.0,1.0,49340.0,49340.0
1,719696,210708,1367858,"2/1/2017 10:38:26,536000",29000.0,784.0,26604.0,120,29529,1,...,0,Samenwonend,Vrouw,Koophuis,41088.0,784.0,1.0,1.0,49340.0,49340.0
2,719696,210708,1367858,"2/1/2017 10:38:26,536000",29000.0,784.0,26604.0,120,29529,1,...,0,Samenwonend,Vrouw,Koophuis,41088.0,784.0,1.0,1.0,49340.0,49340.0
3,719696,210708,1367858,"2/1/2017 10:38:26,536000",29000.0,784.0,26604.0,120,29529,1,...,0,Samenwonend,Vrouw,Koophuis,41088.0,784.0,1.0,1.0,49340.0,49340.0
4,719697,210793,1367861,"2/1/2017 16:08:07,608000",10000.0,NaN,10000.0,44,24958,1,...,0,Alleenstaand,Man,Thuiswonend/ Inwonend,21600.0,708.0,1.0,1.0,235875.0,235875.0


O_SELECTED tells us whetehr an offer is accepted(1)/or refused/cancelled (0)

In [5]:
made_offers = offers[pd.notnull(offers.O_SELECTED)]
made_offers.shape

(118931, 30)

In [6]:
len(made_offers.O_ID.unique())

33401

In [7]:
made_offers = (made_offers.loc[(made_offers.event == "O_Accepted") | (made_offers.event == "O_Cancelled")
                | (made_offers.event == "O_Refused")])

In [8]:
len(made_offers.O_ID.unique()), made_offers.shape

(33047, (33047, 30))

In [9]:
made_offers.head()

,A_ID,O_ID,O_SCENARIO,O_DT_CREATED,O_AMOUNT,O_CREDITSCORE,O_FIRST_WITHDRAWL_AMOUNT,NUMBER_OF_TERMS,O_MONTHLY_COST,O_ACCEPTED,...,NUMBER_OF_CHILDREN,MARITAL_STATUS,SEX,HOUSING_TYPE,INCOMEAMOUNT_YEAR,SCE_CREDITSCORE,SCE_ACCEPTED,SCE_ACCEPTED_OVERRIDE,SCE_MAX_AMOUNT,SCE_MAX_AMOUNT_OVERRIDE
0,719696,210708,1367858,"2/1/2017 10:38:26,536000",29000.0,784.0,26604.0,120,29529,1,...,0,Samenwonend,Vrouw,Koophuis,41088.0,784.0,1.0,1.0,49340.0,49340.0
4,719697,210793,1367861,"2/1/2017 16:08:07,608000",10000.0,NaN,10000.0,44,24958,1,...,0,Alleenstaand,Man,Thuiswonend/ Inwonend,21600.0,708.0,1.0,1.0,235875.0,235875.0
8,719697,210794,1367861,"2/1/2017 16:09:04,325000",13000.0,NaN,13000.0,57,25717,0,...,0,Alleenstaand,Man,Thuiswonend/ Inwonend,21600.0,708.0,1.0,1.0,235875.0,235875.0
12,719698,210961,1367866,"3/1/2017 16:10:34,510000",8000.0,NaN,8000.0,120,9008,1,...,0,Samenwonend,Man,Thuiswonend/ Inwonend,22800.0,805.0,1.0,1.0,8245.0,8245.0
15,719699,210814,1367869,"2/1/2017 17:22:18,492000",5000.0,719.0,175503.0,60,974,1,...,0,Alleenstaand,Vrouw,Huurhuis / Kamers,18000.0,719.0,1.0,1.0,58375.0,58375.0


Attributes:
* Marital Status
* Housing Type
* Income Amount
* Number_Of_Children
* Sex
* Age (2017 - birthyear)
* O_Amount
* Number of terms
* Monthly Cost
* Percentage Loan Amount of Year Income (amount / year_income) * 100 
* Percentage Yearly Cost (12 * Monthly Cost) of Year Income
* Difference between offered amount and iniital requested amount

In [10]:
made_offers["age"]= 2017 - made_offers.BIRTHYEAR
made_offers.head()

,A_ID,O_ID,O_SCENARIO,O_DT_CREATED,O_AMOUNT,O_CREDITSCORE,O_FIRST_WITHDRAWL_AMOUNT,NUMBER_OF_TERMS,O_MONTHLY_COST,O_ACCEPTED,...,MARITAL_STATUS,SEX,HOUSING_TYPE,INCOMEAMOUNT_YEAR,SCE_CREDITSCORE,SCE_ACCEPTED,SCE_ACCEPTED_OVERRIDE,SCE_MAX_AMOUNT,SCE_MAX_AMOUNT_OVERRIDE,age
0,719696,210708,1367858,"2/1/2017 10:38:26,536000",29000.0,784.0,26604.0,120,29529,1,...,Samenwonend,Vrouw,Koophuis,41088.0,784.0,1.0,1.0,49340.0,49340.0,33
4,719697,210793,1367861,"2/1/2017 16:08:07,608000",10000.0,NaN,10000.0,44,24958,1,...,Alleenstaand,Man,Thuiswonend/ Inwonend,21600.0,708.0,1.0,1.0,235875.0,235875.0,22
8,719697,210794,1367861,"2/1/2017 16:09:04,325000",13000.0,NaN,13000.0,57,25717,0,...,Alleenstaand,Man,Thuiswonend/ Inwonend,21600.0,708.0,1.0,1.0,235875.0,235875.0,22
12,719698,210961,1367866,"3/1/2017 16:10:34,510000",8000.0,NaN,8000.0,120,9008,1,...,Samenwonend,Man,Thuiswonend/ Inwonend,22800.0,805.0,1.0,1.0,8245.0,8245.0,28
15,719699,210814,1367869,"2/1/2017 17:22:18,492000",5000.0,719.0,175503.0,60,974,1,...,Alleenstaand,Vrouw,Huurhuis / Kamers,18000.0,719.0,1.0,1.0,58375.0,58375.0,66


In [11]:
made_offers["percentage_amount_income"] = (made_offers.O_AMOUNT / made_offers.INCOMEAMOUNT_YEAR) * 100

In [12]:
made_offers["percentage_cost_income"] = ((made_offers.O_MONTHLY_COST * 12) / made_offers.INCOMEAMOUNT_YEAR) * 100

In [13]:
unique_applications = applications.drop_duplicates(subset = 'A_ID')
unique_applications = unique_applications[['A_ID', 'A_INIT_REQ_AMT', 'A_APPLICATIONTYPE_DESC', 'A_LOANGOALTYPE_DESC']]
unique_applications.head()

,A_ID,A_INIT_REQ_AMT,A_APPLICATIONTYPE_DESC,A_LOANGOALTYPE_DESC
0,719696,29000.0,Nieuw krediet,Auto
8,719697,10000.0,Nieuw krediet,Auto
13,719698,15000.0,Nieuw krediet,Woningverbetering
18,719699,5000.0,Nieuw krediet,Overname lopende leningen
29,719700,5000.0,Nieuw krediet,Niet gespecificeerd


Enrich offer data with initial requested amount (not necesarilly equal to offered amount), application type and loan goal

In [14]:
made_offers = made_offers.merge(unique_applications, on = "A_ID")

Calculate the difference between the offered amount and the initial requested amount

In [15]:
made_offers["diff_req_and_offered_amount"] = abs(made_offers.A_INIT_REQ_AMT - made_offers.O_AMOUNT)

In [16]:
req_amount = tf.feature_column.numeric_column('A_INIT_REQ_AMT')
income = tf.feature_column.numeric_column('INCOMEAMOUNT_YEAR')
number_of_children = tf.feature_column.numeric_column('NUMBER_OF_CHILDREN')
age = tf.feature_column.numeric_column('age')
o_amount = tf.feature_column.numeric_column('O_AMOUNT')
number_of_terms = tf.feature_column.numeric_column('NUMBER_OF_TERMS')
monthly_cost = tf.feature_column.numeric_column('O_MONTHLY_COST')
per_amount_income = tf.feature_column.numeric_column('percentage_amount_income')
per_cost_income = tf.feature_column.numeric_column('percentage_cost_income')
diff_req_and_offered_amount = tf.feature_column.numeric_column("diff_req_and_offered_amount")

In [17]:
made_offers.A_APPLICATIONTYPE_DESC.unique()

array(['Nieuw krediet', 'Limietverhoging'], dtype=object)

In [18]:
made_offers.A_LOANGOALTYPE_DESC.unique()

array(['Auto', 'Woningverbetering', 'Overname lopende leningen',
       'Tourcaravan / camper', 'Anders, zie toelichting', nan,
       'Extra Bestedingsruimte', 'Niet gespecificeerd',
       'Restschuld woning', 'Boot', 'Motor', 'Belastingbetalingen',
       'Zakelijk doel (no go)', 'Schuldsanering (no go)',
       'Verbouwing aan het huis', 'verbouwing meubels/interieur',
       'Verbouwing aan tuin', 'Zakelijk pand (no go)', 'Onderhandse lening'], dtype=object)

In [19]:
made_offers.HOUSING_TYPE.unique()

array(['Koophuis', 'Thuiswonend/ Inwonend', 'Huurhuis / Kamers'], dtype=object)

In [20]:
made_offers.MARITAL_STATUS.unique()

array(['Samenwonend', 'Alleenstaand', 'Gehuwd',
       'Geregistreerd partnerschap'], dtype=object)

In [21]:
loanGoal = tf.feature_column.categorical_column_with_vocabulary_list('A_LOANGOALTYPE_DESC',
                                                                           ['Auto', 'Woningverbetering', 'Overname lopende leningen',
                                                                            'Tourcaravan / camper', 'Anders, zie toelichting', 'nan',
                                                                            'Extra Bestedingsruimte', 'Niet gespecificeerd',
                                                                            'Restschuld woning', 'Boot', 'Motor', 'Belastingbetalingen',
                                                                            'Zakelijk doel (no go)', 'Schuldsanering (no go)',
                                                                            'Verbouwing aan het huis', 'verbouwing meubels/interieur',
                                                                            'Verbouwing aan tuin', 'Zakelijk pand (no go)', 'Onderhandse lening'])
applicationType = tf.feature_column.categorical_column_with_vocabulary_list('A_APPLICATIONTYPE_DESC',
                                                                           ['Nieuw krediet', 'Limietverhoging'])
housingType = tf.feature_column.categorical_column_with_vocabulary_list('HOUSING_TYPE',
                                                                          ['Koophuis', 'Thuiswonend/ Inwonend', 'Huurhuis / Kamers'])
gender = tf.feature_column.categorical_column_with_vocabulary_list('SEX', ['Man', 'Vrouw'])
marital_status = tf.feature_column.categorical_column_with_vocabulary_list('MARITAL_STATUS', ['Samenwonend', 'Alleenstaand',
                                                                                              'Gehuwd', 'Geregistreerd partnerschap'])

In [22]:
feat_cols = [loanGoal, applicationType, req_amount, number_of_terms, monthly_cost, o_amount, housingType, gender, income,
             marital_status, number_of_children, age, per_amount_income, per_cost_income, diff_req_and_offered_amount]

In [23]:
made_offers[['O_SELECTED']] = made_offers[['O_SELECTED']].astype(bool)

In [24]:
made_offers = made_offers[['A_INIT_REQ_AMT', 'INCOMEAMOUNT_YEAR', 'NUMBER_OF_CHILDREN', 'age', 'O_AMOUNT', 'NUMBER_OF_TERMS',
                           'O_MONTHLY_COST', 'percentage_amount_income', 'percentage_cost_income', 'MARITAL_STATUS',
                          "diff_req_and_offered_amount", 'A_LOANGOALTYPE_DESC', 'A_APPLICATIONTYPE_DESC',
                           'HOUSING_TYPE', 'SEX', 'O_SELECTED']]
made_offers = made_offers.dropna(how = "any")
made_offers

,A_INIT_REQ_AMT,INCOMEAMOUNT_YEAR,NUMBER_OF_CHILDREN,age,O_AMOUNT,NUMBER_OF_TERMS,O_MONTHLY_COST,percentage_amount_income,percentage_cost_income,MARITAL_STATUS,diff_req_and_offered_amount,A_LOANGOALTYPE_DESC,A_APPLICATIONTYPE_DESC,HOUSING_TYPE,SEX,O_SELECTED
0,29000.0,41088.0,0,33,29000.0,120,29529,70.580218,862.412383,Samenwonend,0.0,Auto,Nieuw krediet,Koophuis,Vrouw,True
1,10000.0,21600.0,0,22,10000.0,44,24958,46.296296,1386.555556,Alleenstaand,0.0,Auto,Nieuw krediet,Thuiswonend/ Inwonend,Man,False
2,10000.0,21600.0,0,22,13000.0,57,25717,60.185185,1428.722222,Alleenstaand,3000.0,Auto,Nieuw krediet,Thuiswonend/ Inwonend,Man,False
3,15000.0,22800.0,0,28,8000.0,120,9008,35.087719,474.105263,Samenwonend,7000.0,Woningverbetering,Nieuw krediet,Thuiswonend/ Inwonend,Man,False
4,5000.0,18000.0,0,66,5000.0,60,974,27.777778,64.933333,Alleenstaand,0.0,Overname lopende leningen,Nieuw krediet,Huurhuis / Kamers,Vrouw,True
5,5000.0,18000.0,0,66,5000.0,60,974,27.777778,64.933333,Alleenstaand,0.0,Overname lopende leningen,Nieuw krediet,Huurhuis / Kamers,Vrouw,False
6,25000.0,27330.0,0,56,25000.0,120,25456,91.474570,1117.716795,Alleenstaand,0.0,Auto,Nieuw krediet,Huurhuis / Kamers,Man,False
7,25000.0,27330.0,0,56,22000.0,120,22603,80.497622,992.447859,Alleenstaand,3000.0,Auto,Nieuw krediet,Huurhuis / Kamers,Man,True
8,15000.0,19224.0,0,26,13000.0,134,130,67.623804,8.114856,Alleenstaand,2000.0,Auto,Nieuw krediet,Thuiswonend/ Inwonend,Man,True
9,15000.0,30360.0,0,45,15000.0,120,15411,49.407115,609.130435,Gehuwd,0.0,Woningverbetering,Nieuw krediet,Koophuis,Vrouw,False


In [34]:
x_features = made_offers.drop('O_SELECTED', axis = 1)

(31047, 15)

In [26]:
pred_labels = made_offers['O_SELECTED']

In [27]:
x_train, x_test, y_train, y_test = train_test_split(x_features, pred_labels, test_size = 0.2)

In [28]:
input_f = tf.estimator.inputs.pandas_input_fn(x = x_train, y = y_train, batch_size = 100, 
                                              num_epochs = 1000, shuffle = True)

In [29]:
model = tf.estimator.LinearClassifier(feature_columns = feat_cols, n_classes = 2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\s158881\\AppData\\Local\\Temp\\tmp33rsnlwu', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [33]:
model.train(input_fn = input_f, steps = int(1e6))

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from C:\Users\s158881\AppData\Local\Temp\tmp33rsnlwu\model.ckpt-10000
INFO:tensorflow:Saving checkpoints for 10001 into C:\Users\s158881\AppData\Local\Temp\tmp33rsnlwu\model.ckpt.
INFO:tensorflow:loss = 1493.49, step = 10001
INFO:tensorflow:global_step/sec: 303.286
INFO:tensorflow:loss = 8447.46, step = 10101 (0.333 sec)
INFO:tensorflow:global_step/sec: 348.8
INFO:tensorflow:loss = 3367.78, step = 10201 (0.287 sec)
INFO:tensorflow:global_step/sec: 338.191
INFO:tensorflow:loss = 4365.7, step = 10301 (0.296 sec)
INFO:tensorflow:global_step/sec: 328.705
INFO:tensorflow:loss = 7902.85, step = 10401 (0.305 sec)
INFO:tensorflow:global_step/sec: 321.838
INFO:tensorflow:loss = 2296.11, step = 10501 (0.310 sec)
INFO:tensorflow:global_step/sec: 298.748
INFO:tensorflow:loss = 4535.85, step = 10601 (0.335 sec)
INFO:tensorflow:global_step/sec: 310.313
INFO:tensorflow:loss = 2343.07, step = 10701 (0.322 sec)
INFO:tensor

INFO:tensorflow:global_step/sec: 307.009
INFO:tensorflow:loss = 4035.46, step = 18201 (0.325 sec)
INFO:tensorflow:global_step/sec: 324.552
INFO:tensorflow:loss = 1403.32, step = 18301 (0.309 sec)
INFO:tensorflow:global_step/sec: 311.273
INFO:tensorflow:loss = 1068.03, step = 18401 (0.321 sec)
INFO:tensorflow:global_step/sec: 313.223
INFO:tensorflow:loss = 725.854, step = 18501 (0.319 sec)
INFO:tensorflow:global_step/sec: 281.111
INFO:tensorflow:loss = 1792.74, step = 18601 (0.356 sec)
INFO:tensorflow:global_step/sec: 260.236
INFO:tensorflow:loss = 772.84, step = 18701 (0.384 sec)
INFO:tensorflow:global_step/sec: 259.562
INFO:tensorflow:loss = 2234.4, step = 18801 (0.385 sec)
INFO:tensorflow:global_step/sec: 266.839
INFO:tensorflow:loss = 5234.6, step = 18901 (0.375 sec)
INFO:tensorflow:global_step/sec: 312.226
INFO:tensorflow:loss = 6825.95, step = 19001 (0.320 sec)
INFO:tensorflow:global_step/sec: 239.651
INFO:tensorflow:loss = 658.201, step = 19101 (0.419 sec)
INFO:tensorflow:global_

INFO:tensorflow:global_step/sec: 301.908
INFO:tensorflow:loss = 4512.67, step = 26601 (0.331 sec)
INFO:tensorflow:global_step/sec: 303.693
INFO:tensorflow:loss = 1877.96, step = 26701 (0.329 sec)
INFO:tensorflow:global_step/sec: 304.681
INFO:tensorflow:loss = 591.373, step = 26801 (0.328 sec)
INFO:tensorflow:global_step/sec: 314.742
INFO:tensorflow:loss = 5000.06, step = 26901 (0.318 sec)
INFO:tensorflow:global_step/sec: 311.773
INFO:tensorflow:loss = 1245.47, step = 27001 (0.321 sec)
INFO:tensorflow:global_step/sec: 290.076
INFO:tensorflow:loss = 4626.96, step = 27101 (0.344 sec)
INFO:tensorflow:global_step/sec: 296.092
INFO:tensorflow:loss = 2687.64, step = 27201 (0.337 sec)
INFO:tensorflow:global_step/sec: 313.881
INFO:tensorflow:loss = 3677.86, step = 27301 (0.319 sec)
INFO:tensorflow:global_step/sec: 306.709
INFO:tensorflow:loss = 3160.25, step = 27401 (0.326 sec)
INFO:tensorflow:global_step/sec: 316.717
INFO:tensorflow:loss = 4252.16, step = 27501 (0.316 sec)
INFO:tensorflow:glob

INFO:tensorflow:global_step/sec: 300.194
INFO:tensorflow:loss = 3977.81, step = 35001 (0.334 sec)
INFO:tensorflow:global_step/sec: 312.281
INFO:tensorflow:loss = 1196.72, step = 35101 (0.320 sec)
INFO:tensorflow:global_step/sec: 315.753
INFO:tensorflow:loss = 3312.21, step = 35201 (0.316 sec)
INFO:tensorflow:global_step/sec: 313.262
INFO:tensorflow:loss = 3947.58, step = 35301 (0.320 sec)
INFO:tensorflow:global_step/sec: 318.555
INFO:tensorflow:loss = 4574.88, step = 35401 (0.314 sec)
INFO:tensorflow:global_step/sec: 314.611
INFO:tensorflow:loss = 2581.07, step = 35501 (0.318 sec)
INFO:tensorflow:global_step/sec: 304.254
INFO:tensorflow:loss = 2382.35, step = 35601 (0.329 sec)
INFO:tensorflow:global_step/sec: 312.183
INFO:tensorflow:loss = 1202.57, step = 35701 (0.320 sec)
INFO:tensorflow:global_step/sec: 280.813
INFO:tensorflow:loss = 2422.83, step = 35801 (0.356 sec)
INFO:tensorflow:global_step/sec: 253.955
INFO:tensorflow:loss = 3167.34, step = 35901 (0.393 sec)
INFO:tensorflow:glob

INFO:tensorflow:global_step/sec: 296.52
INFO:tensorflow:loss = 2630.75, step = 43401 (0.338 sec)
INFO:tensorflow:global_step/sec: 307.932
INFO:tensorflow:loss = 746.893, step = 43501 (0.324 sec)
INFO:tensorflow:global_step/sec: 296.966
INFO:tensorflow:loss = 3425.53, step = 43601 (0.335 sec)
INFO:tensorflow:global_step/sec: 301.874
INFO:tensorflow:loss = 476.133, step = 43701 (0.333 sec)
INFO:tensorflow:global_step/sec: 304.206
INFO:tensorflow:loss = 2230.17, step = 43801 (0.328 sec)
INFO:tensorflow:global_step/sec: 294.349
INFO:tensorflow:loss = 2743.5, step = 43901 (0.341 sec)
INFO:tensorflow:global_step/sec: 276.045
INFO:tensorflow:loss = 3739.37, step = 44001 (0.362 sec)
INFO:tensorflow:global_step/sec: 307.95
INFO:tensorflow:loss = 374.682, step = 44101 (0.325 sec)
INFO:tensorflow:global_step/sec: 307.467
INFO:tensorflow:loss = 2075.38, step = 44201 (0.326 sec)
INFO:tensorflow:global_step/sec: 311.798
INFO:tensorflow:loss = 1351.3, step = 44301 (0.320 sec)
INFO:tensorflow:global_s

...

INFO:tensorflow:global_step/sec: 273.771
INFO:tensorflow:loss = 976.499, step = 218301 (0.365 sec)
INFO:tensorflow:global_step/sec: 287.151
INFO:tensorflow:loss = 1126.35, step = 218401 (0.348 sec)
INFO:tensorflow:global_step/sec: 265.762
INFO:tensorflow:loss = 639.107, step = 218501 (0.376 sec)
INFO:tensorflow:global_step/sec: 286.732
INFO:tensorflow:loss = 306.489, step = 218601 (0.349 sec)
INFO:tensorflow:global_step/sec: 285.502
INFO:tensorflow:loss = 815.073, step = 218701 (0.350 sec)
INFO:tensorflow:global_step/sec: 298.31
INFO:tensorflow:loss = 1181.35, step = 218801 (0.335 sec)
INFO:tensorflow:global_step/sec: 257.186
INFO:tensorflow:loss = 784.754, step = 218901 (0.389 sec)
INFO:tensorflow:global_step/sec: 285.094
INFO:tensorflow:loss = 1841.26, step = 219001 (0.351 sec)
INFO:tensorflow:global_step/sec: 287.579
INFO:tensorflow:loss = 405.547, step = 219101 (0.348 sec)
INFO:tensorflow:global_step/sec: 293.914
INFO:tensorflow:loss = 1298.17, step = 219201 (0.340 sec)
INFO:tensor

INFO:tensorflow:global_step/sec: 268.217
INFO:tensorflow:loss = 1081.77, step = 226601 (0.373 sec)
INFO:tensorflow:global_step/sec: 283.865
INFO:tensorflow:loss = 228.898, step = 226701 (0.352 sec)
INFO:tensorflow:global_step/sec: 286.743
INFO:tensorflow:loss = 775.07, step = 226801 (0.349 sec)
INFO:tensorflow:global_step/sec: 295.635
INFO:tensorflow:loss = 308.896, step = 226901 (0.338 sec)
INFO:tensorflow:global_step/sec: 288.398
INFO:tensorflow:loss = 964.273, step = 227001 (0.347 sec)
INFO:tensorflow:global_step/sec: 301.448
INFO:tensorflow:loss = 870.037, step = 227101 (0.332 sec)
INFO:tensorflow:global_step/sec: 299.627
INFO:tensorflow:loss = 1308.03, step = 227201 (0.334 sec)
INFO:tensorflow:global_step/sec: 295.65
INFO:tensorflow:loss = 1105.42, step = 227301 (0.339 sec)
INFO:tensorflow:global_step/sec: 300.542
INFO:tensorflow:loss = 774.165, step = 227401 (0.332 sec)
INFO:tensorflow:global_step/sec: 298.75
INFO:tensorflow:loss = 2223.01, step = 227501 (0.335 sec)
INFO:tensorfl

INFO:tensorflow:global_step/sec: 287.138
INFO:tensorflow:loss = 697.086, step = 234901 (0.349 sec)
INFO:tensorflow:global_step/sec: 303.751
INFO:tensorflow:loss = 1309.76, step = 235001 (0.329 sec)
INFO:tensorflow:global_step/sec: 309.354
INFO:tensorflow:loss = 332.633, step = 235101 (0.323 sec)
INFO:tensorflow:global_step/sec: 305.595
INFO:tensorflow:loss = 1185.32, step = 235201 (0.327 sec)
INFO:tensorflow:global_step/sec: 307.951
INFO:tensorflow:loss = 1119.72, step = 235301 (0.325 sec)
INFO:tensorflow:global_step/sec: 294.791
INFO:tensorflow:loss = 1217.75, step = 235401 (0.340 sec)
INFO:tensorflow:global_step/sec: 298.283
INFO:tensorflow:loss = 1328.36, step = 235501 (0.335 sec)
INFO:tensorflow:global_step/sec: 295.653
INFO:tensorflow:loss = 1253.97, step = 235601 (0.338 sec)
INFO:tensorflow:global_step/sec: 291.779
INFO:tensorflow:loss = 487.862, step = 235701 (0.343 sec)
INFO:tensorflow:global_step/sec: 307.471
INFO:tensorflow:loss = 885.174, step = 235801 (0.325 sec)
INFO:tenso

INFO:tensorflow:global_step/sec: 255.888
INFO:tensorflow:loss = 789.889, step = 243201 (0.391 sec)
INFO:tensorflow:global_step/sec: 306.944
INFO:tensorflow:loss = 972.363, step = 243301 (0.326 sec)
INFO:tensorflow:global_step/sec: 304.668
INFO:tensorflow:loss = 1227.3, step = 243401 (0.328 sec)
INFO:tensorflow:global_step/sec: 300.526
INFO:tensorflow:loss = 913.457, step = 243501 (0.333 sec)
INFO:tensorflow:global_step/sec: 301.889
INFO:tensorflow:loss = 1014.97, step = 243601 (0.331 sec)
INFO:tensorflow:global_step/sec: 305.117
INFO:tensorflow:loss = 1117.59, step = 243701 (0.326 sec)
INFO:tensorflow:global_step/sec: 293.482
INFO:tensorflow:loss = 827.556, step = 243801 (0.342 sec)
INFO:tensorflow:global_step/sec: 271.908
INFO:tensorflow:loss = 1370.75, step = 243901 (0.368 sec)
INFO:tensorflow:global_step/sec: 306.547
INFO:tensorflow:loss = 550.727, step = 244001 (0.327 sec)
INFO:tensorflow:global_step/sec: 300.529
INFO:tensorflow:loss = 1033.64, step = 244101 (0.332 sec)
INFO:tensor

INFO:tensorflow:global_step/sec: 281.892
INFO:tensorflow:loss = 712.789, step = 251501 (0.355 sec)
INFO:tensorflow:global_step/sec: 318.758
INFO:tensorflow:loss = 747.818, step = 251601 (0.314 sec)
INFO:tensorflow:global_step/sec: 300.226
INFO:tensorflow:loss = 1008.38, step = 251701 (0.332 sec)
INFO:tensorflow:global_step/sec: 287.158
INFO:tensorflow:loss = 546.112, step = 251801 (0.349 sec)
INFO:tensorflow:global_step/sec: 307.436
INFO:tensorflow:loss = 1947.48, step = 251901 (0.325 sec)
INFO:tensorflow:global_step/sec: 308.415
INFO:tensorflow:loss = 631.381, step = 252001 (0.324 sec)
INFO:tensorflow:global_step/sec: 293.876
INFO:tensorflow:loss = 727.532, step = 252101 (0.341 sec)
INFO:tensorflow:global_step/sec: 292.207
INFO:tensorflow:loss = 719.299, step = 252201 (0.342 sec)
INFO:tensorflow:global_step/sec: 299.615
INFO:tensorflow:loss = 771.906, step = 252301 (0.333 sec)
INFO:tensorflow:global_step/sec: 304.667
INFO:tensorflow:loss = 1488.0, step = 252401 (0.328 sec)
INFO:tensor

In [35]:
evaluation_model_function = tf.estimator.inputs.pandas_input_fn(x = x_test, y = y_test, batch_size = 100,
                                                               num_epochs = 1, shuffle = False)

In [36]:
results = model.evaluate(evaluation_model_function)
results

INFO:tensorflow:Starting evaluation at 2017-12-08-18:16:51
INFO:tensorflow:Restoring parameters from C:\Users\s158881\AppData\Local\Temp\tmp33rsnlwu\model.ckpt-258370
INFO:tensorflow:Finished evaluation at 2017-12-08-18:16:52
INFO:tensorflow:Saving dict for global step 258370: accuracy = 0.542351, accuracy_baseline = 0.556522, auc = 0.544283, auc_precision_recall = 0.634407, average_loss = 1.95587, global_step = 258370, label/mean = 0.556522, loss = 192.793, prediction/mean = 0.589161


{'accuracy': 0.54235107,
 'accuracy_baseline': 0.55652171,
 'auc': 0.54428327,
 'auc_precision_recall': 0.63440669,
 'average_loss': 1.9558743,
 'global_step': 258370,
 'label/mean': 0.55652171,
 'loss': 192.79332,
 'prediction/mean': 0.58916068}